**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [1]:
import re
import gzip

file_name = '../data/dostoevsky-besy-p2.txt.gz'

# загружаем текст ...
with gzip.open(file_name,'rt') as f:  
    text = f.read()[105:] # ...и выкидываем заголовок

print('символов:%i\n'%(len(text)))
print(text[:364].strip())

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.


In [2]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.6.7


In [3]:
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in nltk_sentence_split(text) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text)))

sample(text,2)

предложений: 5556



[['Это', 'в', 'десяти', 'заповедях', '.'],
 ['-', 'Это', 'одна', 'только', 'маска', '.']]

In [7]:
from nltk.lm.preprocessing import padded_everygram_pipeline 

ngram_len_max = 2

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(ngram_len_max, text)

# собираем модель

# from nltk.lm import MLE as Model 
from nltk.lm import Laplace as Model

model = Model(2) 
model.fit(train, vocab)
print(len(model.vocab))

16634


In [8]:
# генерируем продолжения
for sentence in sample(text,30): # выбираем рандомно 10 предложений
    if len(sentence)<10: continue
    # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    # генерируем возможные продолжения
    result = model.generate(3, text_seed=sentence_) 
    print(  ' '.join(sentence_)  + ' ... ' + str( result ) + '\n' )

- В тех же , - коротко ответил Кириллов , тотчас же по голосу угадав о чем спрашивают , и ... ['начав', 'вдруг', 'рассердился']

- Я только лишь взял его в руки и вдруг в одно утро - обыск , арест , полицейский хватает его за шиворот , а вот ... ['день', 'переправил', ',']

Он плюнул и побежал садиться : `` В Скворешники ... [',', 'где', 'он']

Но вы правы , она из-под венца прибежит , стоит вам ... ['об', 'этом', 'роде']

Напротив , не умирайте , а живите ; живите как можно больше , я ... ['по', 'слухам', 'самым']

- Десять шагов между барьерами близко , - заметил ... ['это', 'славянофильская', 'мысль']

Он позора не находит уйти от меня с ... ['места', 'и', ',']

Даже стихи перестал писать , а когда-то и вы забавлялись моими стишками , Николай Всеволодович , ... ['неподвижным', 'взглядом', ',']

Ей хотелось перелить в него свое честолюбие , а он вдруг начал клеить кирку : пастор выходил говорить проповедь , молящиеся слушали , набожно сложив пред собою руки , одна дама утирала платоч

In [9]:
from nltk.util import bigrams

# оцениваем насколько хорошо модель предсказывает слова из датасета
text_bigrams = [ ngram for s in text for ngram in bigrams(s) ]

print( 'perplexity:', model.perplexity( text_bigrams ) )

perplexity: 3660.669736692352
